In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text

# Configuración
sns.set_theme(style="whitegrid")
pd.options.display.float_format = '{:,.2f}'.format

print("🚀 Iniciando Cálculo de Semáforo de Stock...")

# Conexión
DB_USER = 'analista_medhos'
DB_PASS = 'Medhos2025!'
DB_HOST = 'postgres'
DB_NAME = 'medhos_dw'
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/{DB_NAME}')

# =============================================================================
# 1. CARGA DE DATOS
# =============================================================================
print("   📥 Descargando Movimientos Históricos...")
query = """
SELECT cod_insumo, descripcion_insumo, tipo_archivo_detectado, 
       cantidad, fecha_movimiento, deposito_origen_cod
FROM raw_movimientos_siga
WHERE tipo_archivo_detectado IN ('ENTRADA', 'SALIDA')
"""
df = pd.read_sql(query, engine)
df['fecha_movimiento'] = pd.to_datetime(df['fecha_movimiento'])

# =============================================================================
# 2. CÁLCULO DE STOCK ACTUAL (Teórico)
# =============================================================================
print("   🧮 Calculando Balance (Entradas - Salidas)...")

balance = df.groupby(['cod_insumo', 'descripcion_insumo', 'tipo_archivo_detectado'])['cantidad'].sum().unstack(fill_value=0)

if 'ENTRADA' not in balance.columns: balance['ENTRADA'] = 0
if 'SALIDA' not in balance.columns: balance['SALIDA'] = 0

balance['STOCK_ACTUAL'] = balance['ENTRADA'] - balance['SALIDA']
balance['STOCK_ACTUAL'] = balance['STOCK_ACTUAL'].apply(lambda x: max(x, 0)) # Sin stock negativo

print(f"   ✅ Stock calculado para {len(balance)} insumos.")

# =============================================================================
# 3. CÁLCULO DE CPM (CONSUMO PROMEDIO MENSUAL)
# =============================================================================
print("   📉 Calculando Consumo Promedio Mensual (últimos 12 meses)...")

fecha_corte = df['fecha_movimiento'].max() - pd.DateOffset(months=12)
df_ultimo_anio = df[
    (df['tipo_archivo_detectado'] == 'SALIDA') & 
    (df['fecha_movimiento'] >= fecha_corte)
]

consumo_anual = df_ultimo_anio.groupby('cod_insumo')['cantidad'].sum()
cpm = consumo_anual / 12

df_stock = balance.join(cpm.rename("CPM"))
df_stock['CPM'] = df_stock['CPM'].fillna(0)

# =============================================================================
# 4. LÓGICA DEL SEMÁFORO (COBERTURA)
# =============================================================================
def calcular_cobertura(row):
    stock = row['STOCK_ACTUAL']
    cpm = row['CPM']
    
    if cpm <= 0:
        if stock > 0: return 999 # Azul (Hueso)
        else: return 0 # Inactivo
        
    return stock / cpm

df_stock['MESES_COBERTURA'] = df_stock.apply(calcular_cobertura, axis=1)

def semaforo(meses):
    if meses < 1: return 'ROJO'       # Crítico
    elif meses < 3: return 'AMARILLO' # Alerta
    elif meses <= 6: return 'VERDE'   # Saludable
    else: return 'AZUL'               # Sobrestock

df_stock['SEMAFORO'] = df_stock['MESES_COBERTURA'].apply(semaforo)

# =============================================================================
# 5. VALORIZACIÓN Y CRUCE ABC
# =============================================================================
print("   💵 Valorizando Stock y cruzando con clase ABC...")

query_precios = "SELECT cod_insumo, rubro, clase FROM analytics_abc_pareto"
df_abc = pd.read_sql(query_precios, engine).drop_duplicates(subset=['cod_insumo'])

# Flattening (Aplanar) el índice del dataframe de stock
df_stock = df_stock.reset_index()

# Merge
df_final = df_stock.merge(df_abc, on='cod_insumo', how='left')
df_final['rubro'] = df_final['rubro'].fillna('OTROS')
df_final['clase'] = df_final['clase'].fillna('C')

# Costo Unitario Promedio (Dolarizado)
query_unitario = """
SELECT cod_insumo, AVG(precio_unitario / c.tipo_cambio_oficial_venta) as costo_unitario_usd
FROM raw_movimientos_siga r
JOIN dim_cotizaciones c ON r.fecha_movimiento = c.fecha
WHERE r.tipo_archivo_detectado = 'ENTRADA'
GROUP BY cod_insumo
"""
df_costos = pd.read_sql(query_unitario, engine)

df_final = df_final.merge(df_costos, on='cod_insumo', how='left')
df_final['costo_unitario_usd'] = df_final['costo_unitario_usd'].fillna(0)
df_final['VALOR_STOCK_USD'] = df_final['STOCK_ACTUAL'] * df_final['costo_unitario_usd']

# =============================================================================
# 6. LIMPIEZA FINAL Y GUARDADO (FIX CORREGIDO)
# =============================================================================
# Pasamos todo a minúsculas para evitar errores en Postgres
df_final.columns = df_final.columns.str.lower()

print("\n💾 Guardando 'analytics_stock_semaforo' en BD...")
df_final.to_sql('analytics_stock_semaforo', engine, if_exists='replace', index=False)

# Crear índices (Ahora usamos nombres en minúscula)
with engine.begin() as con:
    con.execute(text("CREATE INDEX IF NOT EXISTS idx_stk_rubro ON analytics_stock_semaforo(rubro);"))
    con.execute(text("CREATE INDEX IF NOT EXISTS idx_stk_semaforo ON analytics_stock_semaforo(semaforo);"))
    con.execute(text("CREATE INDEX IF NOT EXISTS idx_stk_clase ON analytics_stock_semaforo(clase);"))

print("✅ Guardado Exitoso.\n")

# =============================================================================
# 7. VISUALIZACIÓN DE ALERTAS
# =============================================================================
# 1. Resumen General
print("🚦 ESTADO DEL INVENTARIO (Cantidad de Insumos):")
print(df_final['semaforo'].value_counts())

# 2. Alerta Crítica (Clase A en Rojo)
# Nota: usamos nombres de columnas en minúscula ahora
criticos = df_final[(df_final['clase'] == 'A') & (df_final['semaforo'] == 'ROJO')]

print(f"\n🔥 ALERTA MÁXIMA: {len(criticos)} insumos CLASE A están en ROJO (Quiebre inminente).")

if len(criticos) > 0:
    print("Top 10 Críticos (Ordenar Compra):")
    cols_mostrar = ['rubro', 'descripcion_insumo', 'stock_actual', 'cpm', 'meses_cobertura']
    display(criticos[cols_mostrar].sort_values('cpm', ascending=False).head(10))

# 3. Dinero Inmovilizado (Azul)
azul = df_final[df_final['semaforo'] == 'AZUL']
total_inmovilizado = azul['valor_stock_usd'].sum()
print(f"\n🥶 DINERO INMOVILIZADO (Sobre-stock > 6 meses): USD {total_inmovilizado:,.2f}")

🚀 Iniciando Cálculo de Semáforo de Stock...
   📥 Descargando Movimientos Históricos...
   🧮 Calculando Balance (Entradas - Salidas)...
   ✅ Stock calculado para 5170 insumos.
   📉 Calculando Consumo Promedio Mensual (últimos 12 meses)...
   💵 Valorizando Stock y cruzando con clase ABC...

💾 Guardando 'analytics_stock_semaforo' en BD...
✅ Guardado Exitoso.

🚦 ESTADO DEL INVENTARIO (Cantidad de Insumos):
semaforo
ROJO        4190
AZUL         763
VERDE        124
AMARILLO      93
Name: count, dtype: int64

🔥 ALERTA MÁXIMA: 343 insumos CLASE A están en ROJO (Quiebre inminente).
Top 10 Críticos (Ordenar Compra):


,rubro,descripcion_insumo,stock_actual,cpm,meses_cobertura
2305,BIOMEDICOS,Aguja hipodermica descartable,0.00,"25,366.67",0.00
4160,BIOMEDICOS,Barbijo descartable de tres pl,0.00,"23,994.17",0.00
2254,BIOMEDICOS,Jeringa descartable 10 cc sin,0.00,"23,102.50",0.00
3355,LABORATORIO,Reactivo p/preanalítica AUTOMA,0.00,"20,625.00",0.00
4520,BIOMEDICOS,Gasa dobladillada 8capas 7x7 p,0.00,"19,113.33",0.00
2014,LABORATORIO,Tubo p/ hemograma c/edta 3ml c,0.00,"15,104.17",0.00
2558,LABORATORIO,Reactivo p/hemograma AUTOMATIZ,0.00,"15,016.67",0.00
2077,LABORATORIO,Reactivo p/glucosa AUTOMATIZAD,"8,437.00","11,733.33",0.72
2080,LABORATORIO,Reactivo p/urea AUTOMATIZADO,0.00,"10,958.33",0.00
1020,MEDICAMENTOS,Solucion cloruro de sodio 0.9%,0.00,"10,554.08",0.00



🥶 DINERO INMOVILIZADO (Sobre-stock > 6 meses): USD 10,810,886.26


In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sqlalchemy import create_engine, text

# Configuración visual
pd.options.display.float_format = '{:,.2f}'.format
pio.renderers.default = "iframe"

print("🚀 Iniciando Cálculo de Semáforo de Stock (V4 - FIX)...")

# Conexión
DB_USER = 'analista_medhos'
DB_PASS = 'Medhos2025!'
DB_HOST = 'postgres'
DB_NAME = 'medhos_dw'
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/{DB_NAME}')

# =============================================================================
# 1. CARGA DE DATOS
# =============================================================================
print("   📥 Descargando Movimientos...")
query = """
SELECT cod_insumo, descripcion_insumo, tipo_archivo_detectado, 
       cantidad, fecha_movimiento, deposito_origen_cod
FROM raw_movimientos_siga
WHERE tipo_archivo_detectado IN ('ENTRADA', 'SALIDA')
"""
df = pd.read_sql(query, engine)
df['fecha_movimiento'] = pd.to_datetime(df['fecha_movimiento'])

# Asegurar que cod_insumo sea string para evitar errores de cruce
df['cod_insumo'] = df['cod_insumo'].astype(str).str.strip()

# =============================================================================
# 2. CÁLCULO DE STOCK ACTUAL
# =============================================================================
print("   🧮 Calculando Balance...")
balance = df.groupby(['cod_insumo', 'descripcion_insumo', 'tipo_archivo_detectado'])['cantidad'].sum().unstack(fill_value=0)

if 'ENTRADA' not in balance.columns: balance['ENTRADA'] = 0
if 'SALIDA' not in balance.columns: balance['SALIDA'] = 0

balance['STOCK_ACTUAL'] = balance['ENTRADA'] - balance['SALIDA']
balance['STOCK_ACTUAL'] = balance['STOCK_ACTUAL'].apply(lambda x: max(x, 0))

# =============================================================================
# 3. CÁLCULO DE CPM
# =============================================================================
print("   📉 Calculando Consumo Promedio Mensual...")
fecha_corte = df['fecha_movimiento'].max() - pd.DateOffset(months=12)
df_ultimo_anio = df[(df['tipo_archivo_detectado'] == 'SALIDA') & (df['fecha_movimiento'] >= fecha_corte)]

cpm = df_ultimo_anio.groupby('cod_insumo')['cantidad'].sum() / 12
df_stock = balance.join(cpm.rename("CPM"))
df_stock['CPM'] = df_stock['CPM'].fillna(0)

# =============================================================================
# 4. LÓGICA DEL SEMÁFORO
# =============================================================================
def calcular_cobertura(row):
    stock = row['STOCK_ACTUAL']
    cpm = row['CPM']
    if cpm <= 0: return 999 if stock > 0 else 0
    return stock / cpm

df_stock['MESES_COBERTURA'] = df_stock.apply(calcular_cobertura, axis=1)

# ETIQUETAS EN MAYÚSCULAS
def semaforo(meses):
    if meses < 1: return 'ROJO'
    elif meses < 3: return 'AMARILLO'
    elif meses <= 6: return 'VERDE'
    else: return 'AZUL'

df_stock['SEMAFORO'] = df_stock['MESES_COBERTURA'].apply(semaforo)

# =============================================================================
# 5. VALORIZACIÓN INTELIGENTE (Cruces Blindados)
# =============================================================================
print("   💵 Valorizando Stock (Prioridad: Precio 2025)...")

# A. Precio 2025
query_2025 = """
SELECT cod_insumo, AVG(precio_unitario / c.tipo_cambio_oficial_venta) as precio_2025
FROM raw_movimientos_siga r
JOIN dim_cotizaciones c ON r.fecha_movimiento = c.fecha
WHERE r.tipo_archivo_detectado = 'ENTRADA' 
AND r.precio_unitario > 0
AND r.fecha_movimiento >= '2025-01-01'
GROUP BY cod_insumo
"""
df_costo_2025 = pd.read_sql(query_2025, engine)
df_costo_2025['cod_insumo'] = df_costo_2025['cod_insumo'].astype(str).str.strip()

# B. Precio Histórico
query_hist = """
SELECT cod_insumo, AVG(precio_unitario / c.tipo_cambio_oficial_venta) as precio_hist
FROM raw_movimientos_siga r
JOIN dim_cotizaciones c ON r.fecha_movimiento = c.fecha
WHERE r.tipo_archivo_detectado = 'ENTRADA' 
AND r.precio_unitario > 0
GROUP BY cod_insumo
"""
df_costo_hist = pd.read_sql(query_hist, engine)
df_costo_hist['cod_insumo'] = df_costo_hist['cod_insumo'].astype(str).str.strip()

# C. Merge
df_stock = df_stock.reset_index()
# Aseguramos tipo string en el dataframe principal también
df_stock['cod_insumo'] = df_stock['cod_insumo'].astype(str).str.strip()

df_final = df_stock.merge(df_costo_2025, on='cod_insumo', how='left')
df_final = df_final.merge(df_costo_hist, on='cod_insumo', how='left')

# Lógica de Precio
df_final['precio_referencia_usd'] = df_final['precio_2025'].fillna(df_final['precio_hist']).fillna(0)
df_final['VALOR_STOCK_USD'] = df_final['STOCK_ACTUAL'] * df_final['precio_referencia_usd']

# D. Agregar Rubros
query_abc = "SELECT cod_insumo, rubro, clase FROM analytics_abc_pareto"
df_abc = pd.read_sql(query_abc, engine).drop_duplicates(subset=['cod_insumo'])
df_abc['cod_insumo'] = df_abc['cod_insumo'].astype(str).str.strip()

df_final = df_final.merge(df_abc, on='cod_insumo', how='left')
df_final['rubro'] = df_final['rubro'].fillna('OTROS')
df_final['clase'] = df_final['clase'].fillna('C')

# Limpieza de columnas a minúsculas para DB
df_final.columns = df_final.columns.str.lower()

# =============================================================================
# 6. GUARDADO EN BD
# =============================================================================
print("\n💾 Guardando 'analytics_stock_semaforo' en BD...")
df_final.to_sql('analytics_stock_semaforo', engine, if_exists='replace', index=False)
with engine.begin() as con:
    con.execute(text("CREATE INDEX IF NOT EXISTS idx_stk_rubro ON analytics_stock_semaforo(rubro);"))
print("✅ Guardado Exitoso.")

# =============================================================================
# 7. ANÁLISIS VISUAL (FIX FILTROS)
# =============================================================================

# A. Gráfico Estado de Salud
print("\n📊 --- 1. ESTADO DE SALUD DEL INVENTARIO (Cantidad de Insumos) ---")
conteo = df_final.groupby(['rubro', 'semaforo']).size().reset_index(name='cantidad')

# CORRECCIÓN: Asegurar que los datos sean strings válidos para Plotly
conteo['rubro'] = conteo['rubro'].astype(str)
conteo['semaforo'] = conteo['semaforo'].astype(str)

fig1 = px.bar(
    conteo, 
    x="rubro", 
    y="cantidad", 
    color="semaforo",
    title="Semaforización de Stock por Rubro",
    color_discrete_map={
        'ROJO': '#e74c3c', 
        'AMARILLO': '#f1c40f', 
        'VERDE': '#2ecc71', 
        'AZUL': '#3498db'
    },
    text_auto=True,
    barmode='group'
)
fig1.update_layout(template="plotly_white", height=500)
fig1.show()

# B. Dinero Inmovilizado (FIX FILTRO)
# CORRECCIÓN: Filtramos por 'AZUL' (Mayúscula) que es el valor real
df_azul = df_final[df_final['semaforo'] == 'AZUL'].groupby('rubro')['valor_stock_usd'].sum().reset_index()
total_azul = df_azul['valor_stock_usd'].sum()

print(f"\n💰 --- 2. DINERO INMOVILIZADO (AZUL > 6 Meses) ---")
print(f"   TOTAL ESTIMADO: USD {total_azul:,.2f}")

fig2 = px.bar(
    df_azul, x="rubro", y="valor_stock_usd",
    title="Capital Inmovilizado por Rubro (USD)",
    text_auto='.2s',
    color="rubro",
    color_discrete_sequence=px.colors.qualitative.Prism
)
fig2.update_layout(template="plotly_white")
fig2.show()

display(df_azul.sort_values('valor_stock_usd', ascending=False).style.format({'valor_stock_usd': '${:,.2f}'}))

# =============================================================================
# 8. ALERTAS CRÍTICAS (TOP 15)
# =============================================================================
print("\n🔥 --- 3. ALERTAS DE QUIEBRE DE STOCK (TOP 15 CLASE A) ---")

rubros = ['MEDICAMENTOS', 'LABORATORIO', 'BIOMEDICOS', 'ODONTOLOGIA']

for r in rubros:
    # CORRECCIÓN: Filtramos 'ROJO' en mayúscula
    criticos = df_final[
        (df_final['rubro'] == r) & 
        (df_final['clase'] == 'A') & 
        (df_final['semaforo'] == 'ROJO') 
    ].sort_values('cpm', ascending=False).head(15)
    
    if len(criticos) > 0:
        print(f"\n🔴 {r} - Top 15 Faltantes Críticos (Stock < 1 mes):")
        cols = ['cod_insumo', 'descripcion_insumo', 'stock_actual', 'cpm', 'meses_cobertura']
        display(criticos[cols].style.format({'cpm': '{:,.1f}', 'stock_actual': '{:,.0f}', 'meses_cobertura': '{:.2f}'}))
    else:
        print(f"\n🟢 {r}: Sin faltantes críticos Clase A.")

🚀 Iniciando Cálculo de Semáforo de Stock (V4 - FIX)...
   📥 Descargando Movimientos...
   🧮 Calculando Balance...
   📉 Calculando Consumo Promedio Mensual...
   💵 Valorizando Stock (Prioridad: Precio 2025)...

💾 Guardando 'analytics_stock_semaforo' en BD...
✅ Guardado Exitoso.

📊 --- 1. ESTADO DE SALUD DEL INVENTARIO (Cantidad de Insumos) ---



💰 --- 2. DINERO INMOVILIZADO (AZUL > 6 Meses) ---
   TOTAL ESTIMADO: USD 4,312,464.73


,rubro,valor_stock_usd
0,BIOMEDICOS,"$3,204,584.96"
2,MEDICAMENTOS,"$788,896.80"
1,LABORATORIO,"$233,024.70"
4,OTROS,"$84,659.12"
3,ODONTOLOGIA,"$1,299.14"



🔥 --- 3. ALERTAS DE QUIEBRE DE STOCK (TOP 15 CLASE A) ---

🔴 MEDICAMENTOS - Top 15 Faltantes Críticos (Stock < 1 mes):


,cod_insumo,descripcion_insumo,stock_actual,cpm,meses_cobertura
1020,25198,Solucion cloruro de sodio 0.9%,0,"10,554.1",0.00
3267,54777,Solucion de cloruro de sodio 0,0,"6,387.8",0.00
779,20911,Propofol 10 mg/ml ampolla x 20,0,"2,636.9",0.00
1016,25180,Solucion de dextrosa 5% en agu,0,"2,283.5",0.00
1021,25200,Solucion de cloruro de sodio 0,0,"1,927.5",0.00
1023,25202,Solucion de ringer lactato x 5,0,"1,289.0",0.00
781,20913,Midazolan 15 mg ampolla x 3 ml,0,991.7,0.00
668,20294,Omeprazol 40 mg ampolla,0,875.0,0.00
192,17873,Agua destilada sachet x 500 ml,0,857.2,0.00
627,20130,Cefazolina 1 gramo ampolla,0,729.0,0.00


...

,cod_insumo,descripcion_insumo,stock_actual,cpm,meses_cobertura
3355,55316,Reactivo p/preanalítica AUTOMA,0,"20,625.0",0.00
2014,39547,Tubo p/ hemograma c/edta 3ml c,0,"15,104.2",0.00
2558,51411,Reactivo p/hemograma AUTOMATIZ,0,"15,016.7",0.00
2077,39838,Reactivo p/glucosa AUTOMATIZAD,"8,437","11,733.3",0.72
2080,39841,Reactivo p/urea AUTOMATIZADO,0,"10,958.3",0.00
2567,51421,Reactivo p/bilirrubina directa,0,"9,662.5",0.00
2073,39832,Reactivo p/GPT AUTOMATIZADO,0,"9,183.3",0.00
2072,39831,Reactivo p/GOT AUTOMATIZADO,0,"9,016.7",0.00
2078,39839,Reactivo p/trigliceridos AUTOM,0,"7,312.5",0.00
2074,39835,Reactivo p/acido urico AUTOMAT,0,"5,033.3",0.00



🔴 BIOMEDICOS - Top 15 Faltantes Críticos (Stock < 1 mes):


,cod_insumo,descripcion_insumo,stock_actual,cpm,meses_cobertura
2305,50252,Aguja hipodermica descartable,0,"25,366.7",0.00
4160,57903,Barbijo descartable de tres pl,0,"23,994.2",0.00
2254,41249,Jeringa descartable 10 cc sin,0,"23,102.5",0.00
4520,58489,Gasa dobladillada 8capas 7x7 p,0,"19,113.3",0.00
2385,50397,Camisolin descartable manga la,0,"9,595.8",0.00
2635,51709,Aposito 10x20 cm 14 gr confec.,0,"8,875.0",0.00
2303,50250,Aguja hipodermica descartable,0,"8,324.6",0.00
2255,41251,Jeringa descartable 5 cc sin a,0,"8,150.0",0.00
4135,57746,Gasa dobladillada a granel 10x,0,"7,750.0",0.00
2386,50398,Camisolin descartable manga la,0,"7,594.7",0.00



🔴 ODONTOLOGIA - Top 15 Faltantes Críticos (Stock < 1 mes):


,cod_insumo,descripcion_insumo,stock_actual,cpm,meses_cobertura
11,1280,Comidas y desayunos de trabajo,0,16.6,0.00
5,1183,Uniformes y Equipos para el Pe,0,16.2,0.00
1710,33591,Papel A4 210 x 297 mm- 80 grs,0,15.8,0.00
2536,51171,FORMULARIO Nº4,0,6.6,0.00
5085,60829,Rodamiento p/ turbina KMD mode,0,3.6,0.00
2654,51747,Lampara LED inalambrica p/foto,0,2.5,0.00
2180,40941,"Cassette p/instrumental 3x20,5",0,0.9,0.00
2961,53288,Drum kit p/impresora Xerox 333,0,0.9,0.00
4001,57340,Toner imp. HP Laser jet 3330 D,0,0.5,0.00
1482,30394,Alquiler Equipos Médicos,0,0.2,0.00
